## Kaggle Homeland Security Competition


In [ ]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32" 

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import cv2
import random
import os
import h5py
#import models
import pandas as pd
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
import datetime

# get package versions
def get_version(*vars):
    for var in vars:
        module = __import__(var)    
        print ('%s: %s' %(var,module.__version__))
    
# package version    
get_version('keras','numpy','matplotlib','cv2','theano')

### Settigns

In [ ]:
# target image size
h,w=330,256
hh,ww=w,h

# Parameter “pre_train” 
# if pre_train=False and  weights.hdf5 does not exist then we perform training from scratch
# if pre_train=False and  weights.hdf5 exists then we load weights and perform prediction only 
# if pre_train=True  and  weights.hdf5 exists then we load weights and continue training
# if pre_train=True  and weights.hdf5 does not exist then it raises an Error!
pre_train=False

# initial learning rate
initLearningRate=4e-4

# normalization 
#norm_type='minus1plus1'
norm_type='zeroMeanUnitStd'

# data folder
path2data='../data/'

# stage 1 training data
path2stage1_train=path2data+'stage1_train.hdf5'

# stage 1 test data
path2stage1_test=path2data+'stage1_test.hdf5'
path2stage2_test=path2data+'stage2_test.hdf5'

# stage 1 sample submission
path2stage1submission = path2data+'stage1_sample_submission.csv'
path2stage2submission = path2data+'stage2_sample_submission.csv'

# stage 1 labels
path2external='/media/mra/win71/data/misc/kaggle'
#path2_stage1_labels=path2external+"/homeland/data/stage1_labels.csv"
path2_stage1_labels=path2data+"stage1_labels.csv"
path2_stage1_solution=path2data+"stage1_solution.csv"



# zone orders in the csv file are like this
zones=['zone1','zone10','zone11','zone12',
       'zone13','zone14','zone15','zone16',
       'zone17','zone2','zone3','zone4',
       'zone5','zone6','zone7','zone8','zone9']

body_zone_desc={
        'zone1' :'Right Bicep',
        'zone10':'Upper left Hip/thigh',
        'zone11':'Lowe Right Thigh',
        'zone12':'Lower left Thigh',
        'zone13':'Right Calf',
        'zone14':'Left Calf(below knee)',
        'zone15':'Right Ankle Bone',
        'zone16':'Left Ankle Bone',
        'zone17':'Upper Back',    
        'zone2':'Right Forearm',
        'zone3':'Left Bicep',
        'zone4':'Left Forearm',
        'zone5':'Upper Chest',
        'zone6':'Right Rib Cage and Abs',
        'zone7':'Left Side Rib Cage and Abs',
        'zone8':'Upper Right Hip/Tigh',
        'zone9':'Groin (Sensetive area)'
        } 

### Utilities

In [ ]:
def array_stats(X):
    X=np.asarray(X)
    print ('array shape: ',X.shape, X.dtype)
    #print 'min: %.3f, max:%.3f, avg: %.3f, std:%.3f' %(np.min(X),np.max(X),np.mean(X),np.std(X))
    print ('min: {}, max: {}, avg: {:.3}, std:{:.3}'.format( np.min(X),np.max(X),np.mean(X),np.std(X)))

# gamma correction        
def gammaAug(images,gamma):
    # images shape: N*C*H*W
    #print ('before gamma:', np.mean(image))
    n1,c1,_,_=images.shape
    for k1 in range(n1):
        #for k2 in range(c1):
        g = (2 * np.random.rand() - 1) * gamma + 1.
        images[k1]=(images[k1]) ** g
    #print ('after gamma:', np.mean(image))
    return images
    
def preprocess(X,xnormType=None):
    if xnormType=='minus1plus1':
        X=X.astype('float32')
        X/=np.max(X)
        X-=0.5
        X=X*2
    elif xnormType=='zeroMeanUnitStd':
        X=X.astype('float32')
        X-=np.mean(X)
        stdX=np.std(X)
        if stdX>0.0:
            X/=stdX
    else:
        print('no normalization!')
    return X
    
# train test model
def train_test_model(X_train,y_train,X_test,y_test,params_train):
    foldnm=params_train['foldnm']  
    pre_train=params_train['pre_train'] 
    batch_size=params_train['batch_size'] 
    augmentation=params_train['augmentation'] 
    gamma=params_train['gamma'] 
    print('batch_size: %s, Augmentation: %s' %(batch_size,augmentation))
    
    print 'fold %s training in progress ...' %foldnm
    # load last weights
    if pre_train== True:
        if  os.path.exists(path2weights):
            model.load_weights(path2weights)
            print 'previous weights loaded!'
        else:
            raise IOError('weights does not exist!!!')
    else:
        if  os.path.exists(path2weights):
            model.load_weights(path2weights)
            print 'previous weights loaded!'
            train_status='previous weights'
            return train_status
    
    # path to csv file to save scores
    path2scorescsv = weightfolder+'/scores.csv'
    first_row = 'train,test'
    with open(path2scorescsv, 'w+') as f:
        f.write(first_row + '\n')
           
    # Fit the model
    start_time=time.time()
    scores_test=[]
    scores_train=[]
    if params_train['loss']=='dice': 
        best_score = 0
        previous_score = 0
    else:
        best_score = 1e6
        previous_score = 1e6
    patience = 0
    
    # convert class vectors to binary class matrices
    #y_train = np_utils.to_categorical(y_train, nb_outputs)
    #y_test = np_utils.to_categorical(y_test, nb_outputs)
    
    
    for epoch in range(params_train['nbepoch']):
    
        print ('epoch: %s,  Current Learning Rate: %.1e' %(epoch,model.optimizer.lr.get_value()))
        #seed = np.random.randint(0, 999999)
    
        #batches=0
        #bs=X_train.shape[0]
        #for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=bs):
            #X_batch=preprocess_input(X_batch)
            #hist = model.fit(X_batch, y_batch,verbose=0,batch_size=batch_size)
            #batches += 1
            #if batches >= len(X_train) / bs:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                #break
            
        #hist=model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),\
                                 #samples_per_epoch=len(X_train), nb_epoch=1, verbose=0)
    
        if augmentation:
            batches=0
            bs=len(X_train)#batch_size*10
            for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=bs,shuffle=False):
                #X_batch=gammaAug(X_batch,gamma)
                X_batch=preprocess(X_batch,norm_type)
                hist=model.fit(X_batch[:,:,np.newaxis], y_batch, batch_size=batch_size,nb_epoch=1, verbose=0)
                batches += 1
                if batches >= len(X_train) / bs:
                    # we need to break the loop by hand because
                    # the generator loops indefinitely
                    break
        else:
            hist=model.fit(preprocess(X_train,norm_type)[:,:,np.newaxis], y_train, batch_size=batch_size,nb_epoch=1, verbose=0)
            
        # evaluate on test and train data
        score_test=model.evaluate(preprocess(X_test,norm_type)[:,:,np.newaxis],y_test,verbose=0,batch_size=batch_size)
        score_train=np.mean(hist.history['loss'])
       
        print ('score_train: %s, score_test: %s' %(score_train,score_test))
        scores_test=np.append(scores_test,score_test)
        scores_train=np.append(scores_train,score_train)    

        # check for improvement    
        if (score_test<=best_score):
            print ("!!!!!!!!!!!!!!!!!!!!!!!!!!! viva, improvement!!!!!!!!!!!!!!!!!!!!!!!!!!!") 
            best_score = score_test
            patience = 0
            model.save_weights(path2weights)  
            model.save(weightfolder+'/model.h5')
            
        # learning rate schedule
        if score_test>previous_score:
            #print "Incrementing Patience."
            patience += 1

        # learning rate schedule                
        if patience == params_train['max_patience']:
            params_train['learning_rate'] = params_train['learning_rate']/2
            print ("Upating Current Learning Rate to: ", params_train['learning_rate'])
            model.optimizer.lr.set_value(params_train['learning_rate'])
            print ("Loading the best weights again. best_score: ",best_score)
            model.load_weights(path2weights)
            patience = 0
        
        # save current test score
        previous_score = score_test    
        
        # store scores into csv file
        with open(path2scorescsv, 'a') as f:
            string = str([score_train,score_test])
            f.write(string + '\n')

        # train test progress plots
        plt.figure(figsize=(10,10))
        plt.plot(scores_test)
        plt.plot(scores_train)
        plt.title('train-validation progress',fontsize=20)
        plt.legend(('test','train'), loc = 'top right',fontsize=20)
        plt.xlabel('epochs',fontsize=20)
        plt.ylabel('loss',fontsize=20)
        plt.grid(True)
        plt.show()
        plt.savefig(weightfolder+'/train_val_progress.png')
            
    
    print ('model was trained!')
    elapsed_time=(time.time()-start_time)/60
    print ('elapsed time: %d  mins' %elapsed_time)      

    # train test progress plots
    plt.figure(figsize=(10,10))
    plt.plot(scores_test)
    plt.plot(scores_train)
    plt.title('train-validation progress',fontsize=20)
    plt.legend(('test','train'), loc = 'top right',fontsize=20)
    plt.xlabel('epochs',fontsize=20)
    plt.ylabel('loss',fontsize=20)
    plt.grid(True)
    plt.show()
    plt.savefig(weightfolder+'/train_val_progress.png')
    
    print 'training completed!'
    train_status='completed!'
    return train_status 

# display a sample subject
def dispSampleSubject(X,y_zone=None): 
    # X shape: N*C*H*W
    sbj_num=np.random.randint(len(X))
    print ('subject: %s' %sbj_num)
    #array_stats(X[sbj_num])

    plt.figure(figsize=(15,15))
    for k in range(16):
        plt.subplot(4,4,k+1)
        plt.imshow(X[sbj_num,k],cmap='gray')
    if y_zone is not None:
        # zones with objects
        nz_label=np.nonzero(y_zone[sbj_num,:])[0]
        for nz_l in nz_label:
            print ('%s: %s' %(zones[nz_l],body_zone_desc[zones[nz_l]]))


In [ ]:
def get_data(data_type='stage1_train'):
    # number of zones
    nb_zones=17

    if data_type=='stage1_train':
        ff=h5py.File(path2stage1_train,'r')
        # get subject ids
        keys=ff.keys()
        ids=np.array(keys[:-1])
        y=ff['labels'].value
    elif data_type=='stage1_leader':
        ff=h5py.File(path2stage1_test,'r') 
        keys=ff.keys()
        ids=np.array(keys)
        y=np.zeros((nb_zones,len(ids)),'uint8')
    elif data_type=='stage2_leader':
        ff=h5py.File(path2stage2_test,'r') 
        keys=ff.keys()
        ids=np.array(keys)
        y=np.zeros((nb_zones,len(ids)),'uint8')
        
        
    X=[]
    for k,id1 in enumerate(ids):
        print k,id1
        X0= ff[id1].value
        X0=cv2.resize(X0,(w,h),interpolation = cv2.INTER_CUBIC)
        X.append(X0)
    X=np.stack(X)
    X=np.transpose(X,(0,3,1,2))
    return X,y.astype('uint8'),ids




# this was to rotate images
def rotateData(data_type,(h,w)):
    path2traintest=path2data+data_type+'_'+str(w)+'by'+str(h)+'.hdf5'
    
    path2traintestR=path2data+data_type+'270'+'_'+str(h)+'by'+str(w)+'.hdf5'

    # load train-test data
    if os.path.exists(path2traintest):
        print ('loading: %s' %path2traintest)
        ff_traintest=h5py.File(path2traintest,'r')
        X=ff_traintest['X']
        try:
            y=ff_traintest['y'].value
        except:
            y=np.zeros(len(X))
        ids=ff_traintest['ids'].value
    else:
        print ('source data does not exist!')
        return
    
    if not os.path.exists(path2traintestR):
        print('wait ...')
        ## rotate
        n,c,w,h=X.shape
        Xr=np.zeros((n,c,h,w),dtype=X.dtype)
        for k in range(n):
            Xr[k]=np.rot90(X[k],k=3,axes=(1,2))
        
        print ('storing: %s' %path2traintestR)
        ff_traintestR=h5py.File(path2traintestR,'w')
        ff_traintestR['X']=Xr
        ff_traintestR['y']=y    
        ff_traintestR['ids']=np.array(ids,'string')    
        ff_traintestR.close()
        print 'hdf5 saved!'
    else:
        print('rotated data exists!')


## loading data

In [ ]:
#datatype='stage1_traintest'
#rotateData(datatype,(h,w))

In [ ]:
# path to train-test.hdf5, first checking if it exists
path2traintest=path2data+'stage1_traintest270_'+str(hh)+'by'+str(ww)+'.hdf5'
if not os.path.exists(path2traintest):
    raise IOError(path2traintest+' data does not exist!')
    #print ('wait ...')
    #X,y,ids=get_data('stage1_train')
    #ff_traintest=h5py.File(path2traintest,'w')
    #ff_traintest['X']=X
    #ff_traintest['y']=y    
    #ff_traintest['ids']=np.array(ids,'string')    
    #ff_traintest.close()
    #print 'hdf5 saved!'

# load train-test data
ff_traintest=h5py.File(path2traintest,'r')
ids=ff_traintest['ids'].value


# read labels
stage1_train=pd.read_csv(path2_stage1_labels)

# labels
nb_zones=17
Probability=stage1_train.Probability
y_zone=np.array(Probability)
y_zone=np.reshape(y_zone,(len(y_zone)/nb_zones,nb_zones))
print 'labels shape:', y_zone.shape


# k-fold cross-validation data split
X=ff_traintest['X']
print ('X, y_zone:', X.shape,y_zone.shape)


In [ ]:
dispSampleSubject(X,y_zone)

### defining AI model

In [ ]:
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D,Dropout
from keras.layers import Activation,Reshape,Permute,Flatten,Dense
from keras.models import Model,Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.optimizers import RMSprop, Adam,Nadam
from keras.layers import GlobalAveragePooling1D,AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers.advanced_activations import ELU,PReLU,LeakyReLU

def conv_block(x_input, num_filters,pool=True,norm=False,drop_rate=0.0,border_mode='same',stride=1):
    x1 = Convolution2D(num_filters,3,3,subsample=(stride,stride),border_mode=border_mode,W_regularizer=l2(1e-4))(x_input)
    if norm:
        x1 = BatchNormalization(mode=2,axis=1)(x1)
        
    if drop_rate > 0.0:
        x1 = GaussianDropout(drop_rate)(x1)

    x1 = LeakyReLU(.1)(x1)
    if pool:
        x1 = MaxPooling2D()(x1)
    x_out = x1
    return x_out


def modelCnnAvg(params):
    # input format: batch_size*timesteps*z*h*w    
    h=params['h']
    w=params['w']
    z=params['z']
    timesteps=params['timesteps']    
    lr=params['learning_rate']
    nb_output=params['nb_outputs']
    loss=params['loss']
    C=params['init_filters']
    border_mode=params['border_mode']
    base_display=params['base_display']
    stride=params['stride']
    dropOutRate=params['dropOutRate']
    optimizer=params['optimizer']
    kS=params['kernelSize']
    kI=params['kernelInit']
    bnEnable=params['batchNormEnable']
    iF=params['init_filters']
    
  
    xin = Input((z,h, w))
    x1 = conv_block(xin,iF,norm=bnEnable,drop_rate=0,stride=stride) 
    x1_ident = AveragePooling2D(pool_size=(2*stride, 2*stride))(xin)
    x1_merged = merge([x1, x1_ident],mode='concat', concat_axis=1)

    x2_1 = conv_block(x1_merged,3*iF,norm=bnEnable,drop_rate=0) 
    x2_1 = conv_block(x2_1,3*iF,norm=bnEnable,pool=False,drop_rate=0) 
    x2_ident = AveragePooling2D()(x1_ident)
    x2_merged = merge([x2_1,x2_ident],mode='concat', concat_axis=1)

    #by branching we reduce the #params
    x3_1 = conv_block(x2_merged,8*iF,norm=bnEnable,drop_rate=0) 
    x3_1 = conv_block(x3_1,8*iF,norm=bnEnable,pool=False,drop_rate=0) 
    x3_ident = AveragePooling2D()(x2_ident)
    x3_merged = merge([x3_1,x3_ident],mode='concat', concat_axis=1)

    x4_1 = conv_block(x3_merged,9*iF,norm=bnEnable,drop_rate=0) 
    x4_1 = conv_block(x4_1,9*iF,norm=bnEnable,pool=False,drop_rate=0) 
    x4_ident = AveragePooling2D()(x3_ident)
    x4_merged = merge([x4_1,x4_ident],mode='concat', concat_axis=1)    

    x5_1 = conv_block(x4_merged,9*iF,norm=bnEnable,pool=False,drop_rate=0) 
    x5_1 = conv_block(x5_1,9*iF,norm=bnEnable,pool=False,drop_rate=0) 
    
    # last layer of encoding    
    x=Flatten() (x5_1)
    
    model_encoder = Model(input=xin, output=x)
    if base_display:
        model_encoder.summary()
    
    model = Sequential()
    model.add(TimeDistributed(model_encoder,input_shape=(timesteps,z,h,w)))    

    
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(dropOutRate))
    model.add(Dense(nb_output, activation='sigmoid',W_regularizer=l2(1e-4)))
    
    if optimizer=='RMSprop':
        optimizer = RMSprop(lr)
    elif optimizer=='Adam':       
        optimizer = Adam(lr)
    elif optimizer=='Nadam':       
        optimizer = Nadam(lr,clipvalue=1.0)        
    model.compile(loss=loss, optimizer=optimizer)

    return model

# random data generator
datagen = ImageDataGenerator(featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        width_shift_range=0.10,
        height_shift_range=0.10,
        shear_range=0.1,
        zoom_range=0.15,
        channel_shift_range=0.0,
        fill_mode='constant',
        cval=0.0,
        horizontal_flip=False,
        vertical_flip=False,
        dim_ordering='th') 



# training params
params_train={
    'h': h, # for models trained on rotated data, swap h and w
    'w': w,
    'z': 1,
    'timesteps':X.shape[1],
    'c':1,           
    'learning_rate': initLearningRate,
    #'optimizer': 'Adam',
    'optimizer': 'Nadam',
    #'loss': 'categorical_crossentropy',
    'loss': 'binary_crossentropy',
    #'loss': 'mean_squared_error',
    'nbepoch': 1000,
    'nb_outputs': 17,
    'init_filters': 24,    
    'max_patience': 30,
    'stride': 2,
    'fully_connected': False,
    'output_activation': 'sigmoid',
    'border_mode':'valid',
    'base_display': True,
    'dropOutRate': 0.5,
    'pre_train': False,
    'foldnm':1,
    'kernelSize':3,
    'kernelInit':'glorot_uniform',
    'batchNormEnable': True,
    'batch_size': 8,
    'gamma': 0,
    'augmentation': True,
}    


model=modelCnnAvg(params_train)
model.summary()

### n-fold train-test

In [ ]:
n_folds=1
#skf = StratifiedKFold(n_splits=5,random_state=123,shuffle=True)
#skf = KFold(n_splits=n_folds,random_state=234)
skf = ShuffleSplit(n_splits=n_folds, test_size=0.1, random_state=321)

# loop over folds
foldnm=0
scores_nfolds=[]

print ('wait ...')
for train_ind, test_ind in skf.split(X,y_zone):
    foldnm+=1    

    train_ind=list(np.sort(train_ind))
    test_ind=list(np.sort(test_ind))
    #print train_ind, train_ind
    #train_ind=train_ind[:10]
    #test_ind=test_ind[:10]
    
    X_train,y_train=X[train_ind],y_zone[train_ind]
    X_test,y_test=X[test_ind],y_zone[test_ind]
    
    array_stats(X_train)
    array_stats(y_train)
    array_stats(X_test)
    array_stats(y_test)
    print ('-'*30)

   # training params
    params_train={
        'h': h,
        'w': w,
        'z': 1,
        'timesteps':X.shape[1],
        'c':1,           
        'learning_rate': initLearningRate,
        #'optimizer': 'Adam',
        'optimizer': 'Nadam',
        #'loss': 'categorical_crossentropy',
        'loss': 'binary_crossentropy',
        #'loss': 'mean_squared_error',
        'nbepoch': 1000,
        'nb_outputs': 17,
        'init_filters': 24,    
        'max_patience': 30,
        'stride': 2,
        'fully_connected': False,
        'output_activation': 'sigmoid',
        'border_mode':'same',
        'base_display': True,
        'dropOutRate': 0.0,
        'pre_train': pre_train,
        'foldnm':foldnm,
        'kernelSize':3,
        'kernelInit':'glorot_uniform',
        'batchNormEnable': True,
        'batch_size': 8,
        'augmentation': True,
        'gamma': 0, # gamma correction
    }    
    
    model=modelCnnAvg(params_train)
    model.summary()
     
    
    # exeriment name to record weights and scores
    netinfo='_cnnSkipStack2GlobalAvg_normZeroMeanUnitVarRotate270'
    experiment='fold'+str(foldnm)+netinfo+'_hw_'+str(h)+'by'+str(w)+'_initfilts_'+str(params_train['init_filters'])
    print ('experiment:', experiment)

    # checkpoint
    weightfolder='./output/weights/'+experiment
    if  not os.path.exists(weightfolder):
        os.makedirs(weightfolder)
    print ('weights folder created')    
    
    # path to weights
    path2weights=weightfolder+"/weights.hdf5"
    path2model=weightfolder+"/model.hdf5"    
    
    # train test on fold #
    train_test_model(X_train,y_train,X_test,y_test,params_train)
    
    # loading best weights from training session
    if  os.path.exists(path2weights):
        model.load_weights(path2weights)
        print 'weights loaded!'
    else:
        raise IOError('weights does not exist!!!')
    #y_test = np_utils.to_categorical(y_test, nb_outputs)
    score_test=model.evaluate(preprocess(X_test,norm_type)[:,:,np.newaxis],y_test,verbose=0,batch_size=8)
    print ('score_test: %.5f' %(score_test))    
    print ('-' *30)
    # store scores for all folds
    scores_nfolds.append(score_test)

print ('average score for %s folds is %s' %(n_folds,np.mean(scores_nfolds)))    

### training progress plot

In [ ]:
def plotScores(weightfolder):
    # path to csv file to save scores
    path2scorescsv = weightfolder+'/scores.csv'
    scores=[]
    with open(path2scorescsv, 'r') as f:
        scores.append(f.read())

    scores=scores[0].split('\n')
    scores=scores[1:] # train test

    scoresTrain=[]
    scoresTest=[]
    for row in scores:

        tmp=row.split(',')
        if len(tmp)>1:
            tmpTrain=float(tmp[0][1:])
            tmpTest=float(tmp[1][1:-1])
            scoresTrain.append(tmpTrain)
            scoresTest.append(tmpTest)
    plt.figure(figsize=(10,5))            
    plt.plot(scoresTrain);
    plt.plot(scoresTest);
    plt.legend(['Train','Test'])
    plt.grid()
plotScores(weightfolder)

### Error Analysis

In [ ]:
model.load_weights(path2weights)
yPred=model.predict(preprocess(X_test,norm_type)[:,:,np.newaxis])
print (yPred.shape)


In [ ]:
def logloss(y_true,y_pred):
    sumY=[]
    n1,n2=y_true.shape
    for i1 in range(n1):
        for i2 in range(n2):
            yi=y_true[i1,i2]
            yih=y_pred[i1,i2]
            # clip outputs
            yi=max(min(yi,1-10**(-15)),10**(-15))
            yih=max(min(yih,1-10**(-15)),10**(-15))
            
            # calculate log loss
            p1=yi*np.log(yih)+(1-yi)*np.log(1-yih)
            sumY.append(p1)
    return -np.mean(sumY)

print 'logloss:',logloss(y_test,yPred)
#print 'logloss:',logloss(y_train,yPredTrain)
print ('-'*30)

print yPred.shape
sumY=[]
errorInds=[]
for i1 in range(len(y_test)):
    for i2 in range(17):
        yi=y_test[i1,i2]
        yi=max(min(yi,1-10**(-15)),10**(-15))
        yih=yPred[i1,i2]
        #if yih<10**(-6):
        #    yih=0.0
        #if yih>0.999:
            #yih=1.0
            
        yih=max(min(yih,1-10**(-15)),10**(-15))
        if (yi>.5) != (yih>.5):
            print i1,i2,yi,yih    
            errorInds.append(i1)
        else:
            p1=yi*np.log(yih)+(1-yi)*np.log(1-yih)
            sumY.append(p1)
print -np.mean(sumY)
print len(sumY)
print ('error indices in test:',errorInds)

### stage 1 leaderboard data

In [ ]:
# rotate data
#data_type='stage1_leader'
#rotateData(data_type,(h,w))

In [ ]:
data_type='stage1_leader270'

# load leaderborad data
path2_stage1leader=path2data+data_type+'_'+str(hh)+'by'+str(ww)+'.hdf5'
if not os.path.exists(path2_stage1leader):
    raise IOerro(path2_stage1leader +' does not exist!')
    #print ('wait ...')
    #X,y,ids=get_data(data_type)
    #ff_stage1leader=h5py.File(path2_stage1leader,'w')
    #ff_stage1leader['X']=X
    #ff_stage1leader['y']=y    
    #ff_stage1leader['ids']=np.array(ids,'string')    
    #ff_stage1leader.close()
    #print 'hdf5 saved!'

# load train-test data
ff_stage1leader=h5py.File(path2_stage1leader,'r')
ids_stage1leader=ff_stage1leader['ids'].value

X_stage1leader=ff_stage1leader['X'].value
#X_stage1leader=preprocess(X_stage1leader,norm_type)
array_stats(X_stage1leader)


In [ ]:
dispSampleSubject(X_stage1leader)

In [ ]:
## evaluate stage 1
yPred1=model.predict(preprocess(X_stage1leader,norm_type)[:,:,np.newaxis])
print (yPred1.shape)

# read labels
stage1_test=pd.read_csv(path2_stage1_solution)
Probability=stage1_test.Probability
y_zone1=np.array(Probability)
y_stage1=np.reshape(y_zone1,(len(y_zone1)/nb_zones,nb_zones))
print 'labels shape:', y_zone1.shape
print 'logloss:',logloss(y_stage1,yPred1)

## stage 2 leaderboard

In [ ]:
# rotate data
#data_type='stage2_leader'
#rotateData(data_type,(h,w))

In [ ]:
data_type='stage2_leader270'

# load leaderborad data
path2_stage2leader=path2data+data_type+'_'+str(hh)+'by'+str(ww)+'.hdf5'
if not os.path.exists(path2_stage2leader):
    raise IOerro(path2_stage2leader +' does not exist!')
    #print ('wait ...')
    #X,y,ids=get_data(data_type)
    #ff_stage2leader=h5py.File(path2_stage1leader,'w')
    #ff_stage2leader['X']=X
    #ff_stage1leader['y']=y    
    #ff_stage2leader['ids']=np.array(ids,'string')    
    #ff_stage2leader.close()
    #print 'hdf5 saved!'

# load train-test data
ff_stage2leader=h5py.File(path2_stage2leader,'r')
ids_stage2leader=ff_stage2leader['ids'].value

X_stage2leader=ff_stage2leader['X'].value
#X_stage1leader=preprocess(X_stage1leader,norm_type)
array_stats(X_stage2leader)


In [ ]:
dispSampleSubject(X_stage2leader)

### predic output

In [ ]:
# prediction for nfolds
y_pred=[]
for foldnm in range(1,n_folds+1):
    
    # load weights
    experiment='fold'+str(foldnm)+netinfo+'_hw_'+str(h)+'by'+str(w)+'_initfilts_'+str(params_train['init_filters'])
    print ('experiment:', experiment)
    weightfolder='./output/weights/'+experiment
    # path to weights
    path2weights=weightfolder+"/weights.hdf5"
    if  os.path.exists(path2weights):
        model.load_weights(path2weights)
        print ('weights loaded!')
    else:
        raise IOError ('weights does not exist!')

    # prediction
    y_pred_perfold=model.predict(preprocess(X_stage2leader,norm_type)[:,:,np.newaxis])
    print y_pred_perfold.shape    
    y_pred.append(y_pred_perfold)        
        
# reshape 
y_pred1=np.array(y_pred)
y_pred2=np.mean(y_pred1,axis=0)

r1,c1=y_pred2.shape
print y_pred2[0:5]
print (r1,c1)
pred=np.reshape(y_pred2,(r1*c1,1))
print (pred.shape)

### create submission

In [ ]:
submission  = pd.read_csv(path2stage2submission)
pid = submission ['Id'].values

# make submission
now = datetime.datetime.now()
info=experiment
suffix = info + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
path2submission = os.path.join('./output/submissions', 'submission_' + suffix + '.csv')

#submission = pd.DataFrame(pred, columns=['Probability'])
submission['Id'] = pid
submission['Probability'] = pred
submission.to_csv(path2submission, index=False)
submission.head()

